In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("/content/diabetes.csv")
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
df.corr()["Outcome"]

,Outcome
Pregnancies,0.221898
Glucose,0.466581
BloodPressure,0.065068
SkinThickness,0.074752
Insulin,0.130548
BMI,0.292695
DiabetesPedigreeFunction,0.173844
Age,0.238356
Outcome,1.000000


In [5]:
X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [7]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [8]:
X = scaler.fit_transform(X)

In [9]:
X.shape

(768, 8)

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size =0.2, random_state = 1 )

In [75]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Dropout

In [18]:
model = Sequential()
model.add(Dense(32, activation="relu", input_dim = 8))
model.add(Dense(1, activation = "sigmoid"))

model.compile(optimizer = "Adam" , loss = "binary_crossentropy", metrics = ["accuracy"])


In [19]:
model.fit(X_train, y_train, batch_size = 32, epochs = 100, validation_data=(X_test, y_test))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.7151 - loss: 0.5902 - val_accuracy: 0.7338 - val_loss: 0.6058
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7155 - loss: 0.5723 - val_accuracy: 0.7338 - val_loss: 0.5795
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7316 - loss: 0.5552 - val_accuracy: 0.7468 - val_loss: 0.5594
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7612 - loss: 0.5234 - val_accuracy: 0.7597 - val_loss: 0.5428
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7776 - loss: 0.4963 - val_accuracy: 0.7662 - val_loss: 0.5322
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7466 - loss: 0.5176 - val_accuracy: 0.7727 - val_loss: 0.5215
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7715 - loss: 0.4868 - val_accuracy: 0.7597 - val_loss: 0.5130
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7590 - loss: 0.4873 - val_accuracy: 0.7532 - 

In [20]:
# 1. How to select appropriate optimizer
# 2. No. of Nodes in a layer
# 3. How to select no. of layers
# 4. All in all one model

In [23]:
pip install -U keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 3.2 MB/s eta 0:00:00


In [24]:
import kerastuner as kt

/tmp/ipython-input-1654478174.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [31]:
def build_model(hp):

  model = Sequential()

  model.add(Dense(32, activation = "relu", input_dim = 8))
  model.add(Dense(1, activation="sigmoid"))

  optimizer= hp.Choice("optimizer", values = ["adam", "sgd", "rmsprop", "adadelta"])

  model.compile(optimizer = optimizer, loss = "binary_crossentropy", metrics = ["accuracy"])

  return model

In [32]:
tuner = kt.RandomSearch(build_model,
                        objective= "val_accuracy",
                        max_trials=5)

Reloading Tuner from ./untitled_project/tuner0.json


In [33]:
tuner.search(X_train, y_train, epochs = 5, validation_data = (X_test, y_test))

Trial 4 Complete [00h 00m 03s]
val_accuracy: 0.7142857313156128

Best val_accuracy So Far: 0.7532467246055603
Total elapsed time: 00h 01m 34s


In [35]:
tuner.get_best_hyperparameters()[0].values


{'optimizer': 'rmsprop'}

In [36]:
model = tuner.get_best_models(num_models =1)[0]

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [37]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [38]:
model.fit(X_train, y_train, batch_size = 32, epochs = 100,initial_epoch=6, validation_data=(X_test, y_test) )

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - accuracy: 0.7351 - loss: 0.5372 - val_accuracy: 0.7403 - val_loss: 0.5156
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7455 - loss: 0.5098 - val_accuracy: 0.7468 - val_loss: 0.5068
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7353 - loss: 0.5232 - val_accuracy: 0.7662 - val_loss: 0.4977
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7806 - loss: 0.4774 - val_accuracy: 0.7662 - val_loss: 0.4919
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7630 - loss: 0.5005 - val_accuracy: 0.7662 - val_loss: 0.4860
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7622 - loss: 0.4876 - val_accuracy: 0.7727 - val_loss: 0.4822
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7620 - loss: 0.4705 - val_accuracy: 0.7792 - val_loss: 0.4800
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7648 - loss: 0.4659 - val_accuracy: 0.79

In [48]:
def build_model(hp):
  model = Sequential()

  units = hp.Int("units", min_value=8, max_value=128, step = 8)

  model.add(Dense(units = units, activation="relu", input_dim = 8))
  model.add(Dense(1, activation="sigmoid"))

  model.compile(optimizer="rmsprop", loss = "binary_crossentropy", metrics = ["accuracy"])

  return model



In [49]:
tuner = kt.RandomSearch(build_model,
                        objective = "val_accuracy",
                        max_trials =5,
                        directory = "mydir",
                        project_name = "Kashish")

In [50]:
tuner.search(X_train, y_train, epochs = 5, validation_data = (X_test, y_test) )

Trial 5 Complete [00h 00m 03s]
val_accuracy: 0.7467532753944397

Best val_accuracy So Far: 0.798701286315918
Total elapsed time: 00h 00m 13s


In [51]:
tuner.get_best_hyperparameters()[0].values

{'units': 88}

In [52]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [53]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 88)             │           792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            89 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 881 (3.44 KB)

 Trainable params: 881 (3.44 KB)

 Non-trainable params: 0 (0.00 B)

In [55]:
model.fit(X_train, y_train, batch_size =32, epochs = 100, initial_epoch=6 )

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7679 - loss: 0.5076   
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7528 - loss: 0.4947 
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7536 - loss: 0.4801 
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7466 - loss: 0.4785 
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7592 - loss: 0.4707 
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7549 - loss: 0.4695 
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7798 - loss: 0.4403 
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7696 - loss: 0.4441 
Epoch 15/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8055 - loss: 0.4153 
Epoch 16/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7666 - loss: 0.4512 
Epoch 17/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7823 - loss: 0.4361 
Epoch 18/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3

In [58]:
def build_model(hp):

  model = Sequential()

  model.add(Dense(88, activation= "relu", input_dim = 8))

  for i in range(hp.Int("num_layeres", min_value =1, max_value = 10)):

    model.add(Dense(88, activation="relu"))

  model.add(Dense(1, activation="sigmoid"))

  model.compile(optimizer="rmsprop", loss = "binary_crossentropy", metrics = ["accuracy"])

  return model

In [59]:
tuner = kt.RandomSearch(build_model,
                        objective = "val_accuracy",
                        max_trials = 3,
                        directory = "mydir",
                        project_name = "num_layers")

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [60]:
tuner.search(X_train, y_train, epochs = 5, validation_data = (X_test, y_test))

Trial 3 Complete [00h 00m 03s]
val_accuracy: 0.798701286315918

Best val_accuracy So Far: 0.8116883039474487
Total elapsed time: 00h 00m 11s


In [61]:
tuner.get_best_hyperparameters()[0].values

{'num_layeres': 5}

In [62]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 16 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [63]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 88)             │           792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 88)             │         7,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 88)             │         7,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 88)             │         7,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 88)             │         7,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 88)             │         7,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 1)              │            89 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 40,041 (156.41 KB)

 Trainable params: 40,041 (156.41 KB)

 Non-trainable params: 0 (0.00 B)

In [64]:
model.fit(X_train, y_train, epochs = 100, initial_epoch=6, validation_data=(X_test, y_test) )

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 30ms/step - accuracy: 0.8061 - loss: 0.4400 - val_accuracy: 0.7922 - val_loss: 0.4752
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7688 - loss: 0.4548 - val_accuracy: 0.7792 - val_loss: 0.4981
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7896 - loss: 0.4229 - val_accuracy: 0.7857 - val_loss: 0.4847
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8274 - loss: 0.4014 - val_accuracy: 0.7987 - val_loss: 0.4833
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.8251 - loss: 0.3889 - val_accuracy: 0.7662 - val_loss: 0.4835
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8198 - loss: 0.3844 - val_accuracy: 0.7727 - val_loss: 0.5418
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8201 - loss: 0.3810 - val_accuracy: 0.7597 - val_loss: 0.5141
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8432 - loss: 0.3564 - val_accuracy: 

ALL IN ONE

In [76]:
def build_model(hp):

  model = Sequential()

  counter = 0


  for i in range(hp.Int("num_layers", min_value=1, max_value=10)):

    if counter == 0:

      model.add(
          Dense(
              hp.Int("units" + str(i), min_value=8, max_value=128, step =8),
              activation= hp.Choice("activation" + str(i), values=["relu","tanh","sigmoid"]),
              input_dim=8
              )
          )
      model.add(Dropout(hp.Choice("Dropout" + str(i), values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    else:

      model.add(
          Dense(
              hp.Int("units" + str(i), min_value=8, max_value=128, step =8),
              activation= hp.Choice("activation" + str(i), values=["relu","tanh","sigmoid"]),
              )
          )
      model.add(Dropout(hp.Choice("Dropout" + str(i), values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    counter+=1

  model.add(Dense(1, activation= "sigmoid"))

  model.compile(optimizer= hp.Choice("optimizer", values=["rmsprop", "sgd", "adam"]),
                loss = "binary_crossentropy",
                metrics = ["accuracy"])


  return model

In [77]:
tuner = kt.RandomSearch(build_model,
                        objective = "val_accuracy",
                        max_trials=3,
                        directory="mydir",
                        project_name="AllinOnewithDropout")

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [78]:
tuner.search(X_train, y_train,epochs=5, validation_data=(X_test, y_test))

Trial 3 Complete [00h 00m 04s]
val_accuracy: 0.6428571343421936

Best val_accuracy So Far: 0.7402597665786743
Total elapsed time: 00h 00m 12s


In [79]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 1,
 'units0': 32,
 'activation0': 'relu',
 'Dropout0': 0.4,
 'optimizer': 'adam'}

In [80]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [81]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [82]:
model.fit(X_train, y_train, epochs= 200, initial_epoch=6, validation_data=(X_test, y_test))

Epoch 7/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.6971 - loss: 0.6034 - val_accuracy: 0.7078 - val_loss: 0.5696
Epoch 8/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6850 - loss: 0.5869 - val_accuracy: 0.7143 - val_loss: 0.5478
Epoch 9/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7112 - loss: 0.5623 - val_accuracy: 0.7532 - val_loss: 0.5323
Epoch 10/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7370 - loss: 0.5553 - val_accuracy: 0.7532 - val_loss: 0.5201
Epoch 11/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7285 - loss: 0.5289 - val_accuracy: 0.7857 - val_loss: 0.5118
Epoch 12/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7164 - loss: 0.5420 - val_accuracy: 0.7922 - val_loss: 0.5056
Epoch 13/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7163 - loss: 0.5424 - val_accuracy: 0.7922 - val_loss: 0.4995
Epoch 14/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7304 - loss: 0.5426 - val_accuracy: 0.78